In [ ]:
def extractor(cp):
    sport=None
    plen=None
    nRw={}
    try:
        dport=cp['IP'].dport
        plen=cp['IP'].len
        if plen>60 and(dport==8000):
            try:
                nRw['src']=cp['IP'].src
                nRw['dst']=cp['IP'].dst
                nRw['id']=cp['IP'].id
                nRw['ihl']=cp['IP'].ihl
                nRw['plen']=cp['IP'].len
                nRw['seq']=cp['IP'].seq
                nRw['ttl']=cp['IP'].ttl
                nRw['ack']=cp['IP'].ack
                nRw['window']=cp['IP'].window
                nRw['dport']=dport
                nRw['sport']=cp['IP'].sport
                nRw['stime']=cp.time
                nRw['etime']=cp.time
                nRw['label']=1
                nRw['pcount']=1
                nRw['honeypot']=0
                nRw['wronglogin']=0
                nRw['avgpacket']=0
                rawData = cp['Raw'].load
                load = str(cp['Raw'].load)
                if load.find('empadmlgn.html')>=0:
                    print('Honeypot detected brute force attack from ',nRw['src'])
                    nRw['label']=0
                    nRw['honeypot']=1
                elif load.find('login.html')>=0:                    
                    nRw['wronglogin']=1
            except: pass
            collection(nRw)
    except BaseException as err:
        pass
    return
def collection(rw):
    global npds, pcount, tot
    if(tot%100==0):
        print("Packet count: ",tot)
    tot=tot+1
    if((npds['src'].eq(rw['src'])).any()):
        rwcount = len(npds[npds['src'] == rw['src']])
        ind = npds.index[(npds['src'] == rw['src'])].tolist()
        rw['pcount']=rwcount+1
        rw['stime']=npds.at[ind[0],'stime']
        timediff=round(rw['etime']-rw['stime'])
        rw['ttime']=timediff
        avgpacket=rwcount/timediff
        rw['avgpacket']=avgpacket
        if(avgpacket>=3 and rwcount>10):
            rw['label']=0
            #print('DDoS attack detected from ',rw['src'])
        if rw['wronglogin']==1:
            rw['wronglogin']=npds.at[ind[0],'wronglogin']+1
            if rw['wronglogin']>5:
                rw['label']=0
        npds = npds.append(rw , ignore_index=True)
    else:
        npds = npds.append(rw , ignore_index=True)
        pcount=pcount+1
        print('npds: ',count,rw['src'])

import numpy as np
import pandas as pd
from scapy.all import *
pcount=0
tot=1
npds = pd.DataFrame(columns=['src','dst','sport','dport','id','stime','etime','pcount','plen','ihl','ttl','seq','ack','window','ttime','avgpacket','wronglogin','honeypot','label'])
sniff(prn = extractor, store=0, count=900000)